<a href="https://colab.research.google.com/github/hahaeaezz-arch/chaemin-lee/blob/main/%EB%94%A5%EB%9F%AC%EB%8B%9D_%EA%B3%BC%EC%A0%9C1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

train = pd.read_csv("/content/drive/MyDrive/딥러닝_과제 모음/건포도 이름 맞추기/train.csv")
test = pd.read_csv("/content/drive/MyDrive/딥러닝_과제 모음/건포도 이름 맞추기/test.csv")

X = train.drop(columns=["Class"])
y = train["Class"]

# Label encode the target variable 'y'
le = LabelEncoder()
y_encoded = le.fit_transform(y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
test_scaled = scaler.transform(test)

X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

es = EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=300, batch_size=16, callbacks=[es], verbose=0)

val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
print(f"Validation Accuracy: {val_acc:.5f}")

y_pred_test = (model.predict(test_scaled) > 0.5).astype(int).flatten()

submission = pd.read_csv("/content/drive/MyDrive/딥러닝_과제 모음/건포도 이름 맞추기/submission.csv")
submission.dropna(axis=1, inplace=True)
submission["Class"] = le.inverse_transform(y_pred_test) # Inverse transform to get original labels
submission.to_csv("new_submission.csv", index=False)

print("new_submission.csv 파일 생성 완료!")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Validation Accuracy: 0.89583
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
new_submission.csv 파일 생성 완료!
